In [1]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

In [2]:
df2 = pd.read_csv(r'D:\Szakdolgozat_data\Features_jupyter\df2.csv')
df2['SHOT_MADE'] = df2['SHOT_MADE_FLAG']
df3 = df2.drop_duplicates(subset=['SHOOTER_NAME'])
df4 = df2

df4['SHOT_ZONE'] = 'E: 3Pointer'
df4['SHOT_ATTEMPTED'] = 1
df4['SHOT_MISSED'] = df4['SHOT_ATTEMPTED'] - df4['SHOT_MADE']
#df4['Success_rate'] = df4['SHOT_MADE'] / df4['SHOT_ATTEMPTED']

df4['SHOT_ZONE'] = np.where(df4['SHOT_DISTANCE']<22, 'D: 16ft-3P',df4['SHOT_ZONE'])
df4['SHOT_ZONE'] = np.where(df4['SHOT_DISTANCE']<16, 'C: 10-16ft',df4['SHOT_ZONE'])
df4['SHOT_ZONE'] = np.where(df4['SHOT_DISTANCE']<10, 'B: 3-10ft',df4['SHOT_ZONE'])
df4['SHOT_ZONE'] = np.where(df4['SHOT_DISTANCE']<3, 'A: 0-3ft',df4['SHOT_ZONE'])
df4 = df4.groupby(['Shooter_Team_Name','SHOT_ZONE',], as_index = False)[['SHOT_ATTEMPTED','SHOT_MADE','SHOT_MISSED']].sum()
df4
#list(df2.columns)

,Shooter_Team_Name,SHOT_ZONE,SHOT_ATTEMPTED,SHOT_MADE,SHOT_MISSED
0,Atlanta Hawks,A: 0-3ft,98,49,49
1,Atlanta Hawks,B: 3-10ft,31,7,24
2,Atlanta Hawks,C: 10-16ft,24,9,15
3,Atlanta Hawks,D: 16ft-3P,28,10,18
4,Atlanta Hawks,E: 3Pointer,67,19,48
...,...,...,...,...,...
145,Washington Wizards,A: 0-3ft,79,45,34
146,Washington Wizards,B: 3-10ft,23,9,14
147,Washington Wizards,C: 10-16ft,40,14,26
148,Washington Wizards,D: 16ft-3P,51,14,37


In [3]:
app = JupyterDash(__name__)

#fig = px.bar(df_bar, x="Fruit", y="Amount", color="City", barmode="group")


team1_dropdown = dcc.Dropdown(options=df2['Shooter_Team_Name'].unique())
team2_dropdown = dcc.Dropdown(options=df2['Shooter_Team_Name'].unique())
player_dropdown = dcc.Dropdown(options=df2['SHOOTER_NAME'].unique())

app.layout = html.Div([
    html.Div(children=[
    html.H1(children='Team shots graph'),
    team1_dropdown,
    team2_dropdown,
    html.Div(children=[
    dcc.Graph(id='team_shot-graph_1',style={'width': '50%','display': 'inline-block'}),
    dcc.Graph(id='team_shot-graph_2',style={'width': '50%','display': 'inline-block'})
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div(children=[
    html.H1(children='Player graph'),
    player_dropdown,
    dcc.Graph(id='player-graph')  
    ])
])
    ])
# Callback function that automatically updates the tip-graph based on chosen colorscale
@app.callback(
    Output(component_id='team_shot-graph_1', component_property='figure'),
    Output(component_id='team_shot-graph_2', component_property='figure'),
    Output(component_id='player-graph', component_property='figure'),
    Input(component_id=team1_dropdown, component_property='value'),
    Input(component_id=team2_dropdown, component_property='value'),
    Input(component_id=player_dropdown, component_property='value')
)

def update_graph(selected_team1, selected_team2, selected_player):
    filtered_df4 = df4[df4['Shooter_Team_Name'] == selected_team1 ]
    filtered_df4_2 = df4[df4['Shooter_Team_Name'] == selected_team2]
    filtered_df3 = df2[df2['SHOOTER_NAME'] == selected_player]
    
    
    fig = px.bar((filtered_df4),
                       x='SHOT_ZONE', y=['SHOT_MISSED','SHOT_MADE'], labels = {'x':'SHOT_ZONE','y':"Number of shots"},
                       #color='SHOT_MADE_FLAG', hover_name="SHOOTER_NAME", color_continuous_scale=["red", "lightgreen"],
                       title=f'Shots of {selected_team1}:')
    fig2 = px.bar((filtered_df4_2),
                       x='SHOT_ZONE', y=['SHOT_MISSED','SHOT_MADE'], labels = {'x':'SHOT_ZONE','y':"Number of shots"},
                       #color='SHOT_MADE_FLAG', hover_name="SHOOTER_NAME", color_continuous_scale=["red", "lightgreen"],
                       title=f'Shots of {selected_team2}:')
    
    fig3 = px.scatter(filtered_df3,
                       x='SHOT_DISTANCE', y='CLOSEST_DEFENDER_DIST', labels = {'SHOT_DISTANCE':'Shot distance (ft)','CLOSEST_DEFENDER_DIST':"Distance of closest defender (ft)"},
                       color = 'SHOT_MADE',color_continuous_scale=["red", "lightgreen"],
                       title=f'Shots of {selected_player}')
    
    fig.update_layout(yaxis_range=[0,120])
    fig2.update_layout(yaxis_range=[0,120])

    
    return fig, fig2, fig3

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [3]:
app = JupyterDash(__name__)

#fig = px.bar(df_bar, x="Fruit", y="Amount", color="City", barmode="group")


team_dropdown = dcc.Dropdown(options=df2['SHOOTER_TEAM_ID'].unique())
player_dropdown = dcc.Dropdown(options=df2['SHOOTER_NAME'].unique())

app.layout = html.Div([
    html.Div(children=[
    html.H1(children='Team shots graph'),
    team_dropdown,
    dcc.Graph(id='team_shot-graph')  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div(children=[
    html.H1(children='Player graph'),
    player_dropdown,
    dcc.Graph(id='player-graph')  
    ])
    #Div for third
    html.Div(children=[
    html.H1(children='Player percentages'),
    player_perc_dropdown,
    dcc.Graph(id='player-perc-graph')  
    ])
    
    
    
    fig = go.Figure(data=[go.Bar(
    x=['Feature A', 'Feature B', 'Feature C',
       'Feature D', 'Feature E'],
    y=[20, 14, 23, 25, 22],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text='Least Used Feature')
])
# Callback function that automatically updates the tip-graph based on chosen colorscale
@app.callback(
    Output(component_id='team_shot-graph', component_property='figure'),
    Output(component_id='player-graph', component_property='figure'),
    Output(component_id='player-perc-graph', component_property='figure'),
    Input(component_id=team_dropdown, component_property='value'),
    Input(component_id=player_dropdown, component_property='value'),
    Input(component_id=player_perc_dropdown, component_property='value')
)

def update_graph(selected_team, selected_player, selected_shooter):
    filtered_df2 = df2[df2['SHOOTER_TEAM_ID'] == selected_team]
    filtered_df2_2 = df2[df2['SHOOTER_NAME'] == selected_player]
    filtered_df3 = df3[df3['SHOOTER_NAME']]
    fig = px.scatter(filtered_df2,
                       x='SHOT_DISTANCE', y='FG%_imp',
                       color='SHOT_MADE_FLAG', hover_name="SHOOTER_NAME", color_continuous_scale=["red", "lightgreen"],
                       title=f'Shots of team: {selected_team}')
    fig2 = px.scatter(filtered_df2_2,
                       x='SHOT_DISTANCE', y='CLOSEST_DEFENDER_DIST',
                       color='SHOT_MADE_FLAG', color_continuous_scale=["red", "lightgreen"],
                       title=f'Shots of {selected_player}')
    fig3 = px.bar((filtered_df3, x=['FG% from 2P','FG% from 0-3','FG% from 3-10','FG% from 10-16','FG% from 16-3P','FG% from 3P'],
                   y="Amount", color="City", barmode="group"))
    return fig, fig2

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [3]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import pandas as pd

app = JupyterDash(__name__)

#df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
team_dropdown = dcc.Dropdown(options=df2['SHOOTER_TEAM_ID'].unique())

app.layout = html.Div(children=[
    html.H1(children='Team shots graph'),
    team_dropdown,
    dcc.Graph(id='team_shot-graph')
])
@app.callback(
    Output(component_id='team_shot-graph', component_property='figure'),
    Input(component_id=team_dropdown, component_property='value')
)
def update_graph(selected_team):
    filtered_df2 = df2[df2['SHOOTER_TEAM_ID'] == selected_team]
    fig = px.scatter(filtered_df2,
                       x='SHOT_DISTANCE', y='FG%_imp',
                       color='SHOT_MADE_FLAG', hover_name="SHOOTER_NAME",
                       title=f'Shots of team: {selected_team}')
    return fig



if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/


In [3]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


app = JupyterDash(__name__)


fig = px.scatter(df2, x="FG%_imp", y="SHOT_DISTANCE",
                  color="SHOOTER_TEAM_ID", hover_name="SHOOTER_NAME",
                 log_x=True, size_max=60)

app.layout = html.Div([
    dcc.Graph(
        id='distance_and_makes',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [ ]:
df3 = df2.head(45)


app = JupyterDash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})



#fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")
fig = px.bar(df3, x ='MINUTES_REMAINING', y = 'SHOT_DISTANCE')#, color = 'TEAM_ID')

app.layout = html.Div(children=[
    html.H1(children='Hello Dash_proba'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
#______________________________________DONE ALREADY___________________________________________________

In [5]:
allstat = pd.read_csv(r'D:\Szakdolgozat_data\Features_jupyter\all_stats_added.csv')
imputed = pd.read_csv(r'D:\Szakdolgozat_data\Features_jupyter\after_imputation.csv')
imputed

,Unnamed: 0.1,Unnamed: 0,SHOT_DISTANCE,2P%,3P%,True Shooting,CLOSEST_DEFENDER_DIST,On_ball_defender_dist_to_ball,Screener_defender_dist_to_ball,On_ball_defender_dist_to_ball_plus25,...,FG%_P9,% of shots 3P_P9,2P%_P9,3P%_P9,True Shooting_P9,FG%_P10,2P%_P10,3P%_P10,True Shooting_P10,points_scored
0,0,0,13.0,0.487,0.358,0.520,2.588160,13.555883,22.978021,12.698297,...,0.401,0.457,0.449,0.345,0.501,0.514,0.515,0.000,0.504,2.0
1,1,1,12.0,0.513,0.356,0.565,5.626314,13.924417,25.712309,21.429576,...,0.401,0.457,0.449,0.345,0.501,0.514,0.515,0.000,0.504,2.0
2,2,2,8.0,0.449,0.345,0.501,2.137450,16.916208,27.552834,10.159932,...,0.401,0.457,0.449,0.345,0.501,0.514,0.515,0.000,0.504,2.0
3,3,3,20.0,0.547,0.306,0.563,5.997132,15.362563,15.757765,12.552773,...,0.401,0.457,0.449,0.345,0.501,0.514,0.515,0.000,0.504,0.0
4,4,4,8.0,0.515,0.000,0.504,2.367214,11.087272,19.341080,10.466913,...,0.401,0.457,0.449,0.345,0.501,0.514,0.515,0.000,0.504,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,5535,5535,24.0,0.534,0.366,0.569,9.475067,15.152548,15.761466,11.801533,...,0.514,0.002,0.515,0.000,0.504,0.353,0.398,0.292,0.436,3.0
5536,5536,5536,2.0,0.515,0.281,0.557,2.385489,24.708539,29.008397,10.646605,...,0.514,0.002,0.515,0.000,0.504,0.353,0.398,0.292,0.436,2.0
5537,5537,5537,2.0,0.515,0.281,0.557,2.385489,11.365732,14.854601,3.781121,...,0.514,0.002,0.515,0.000,0.504,0.353,0.398,0.292,0.436,2.0
5538,5538,5538,28.0,0.477,0.299,0.511,5.055592,18.248456,23.793854,15.743521,...,0.514,0.002,0.515,0.000,0.504,0.353,0.398,0.292,0.436,0.0


In [14]:
allstat

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Game_ID,Event_Number,Frame_Number,game_clock_x,shot_clock_x,quarter_x,...,FT%_P10,True Shooting_P10,AST%_P10,OWS_P10,DWS_P10,WS/48_P10,OBPM_P10,DBPM_P10,Height_P10,Weight_P10
0,0,0,0,0,21500001,5,315,689.51,16.02,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
1,1,1,1,1,21500001,6,320,670.46,15.60,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
2,2,2,2,2,21500001,7,296,650.49,16.09,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
3,3,3,3,3,21500001,8,296,635.54,15.10,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
4,4,5,5,5,21500001,10,142,620.75,18.71,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,5535,16797,16797,16797,21500151,452,119,622.14,11.43,4,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5536,5536,16800,16800,16800,21500151,466,108,557.10,14.53,4,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5537,5537,16801,16801,16801,21500151,466,216,552.78,10.00,4,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5538,5538,16802,16802,16802,21500151,467,108,538.11,12.71,4,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040


In [18]:
numericstat = allstat[['FG%','Avg dist','% of shots 2P','% of shots 0-3','% of shots 3-10','% of shots 10-16','% of shots 16-3P','% of shots 3P','FG% from 2P','FG% from 0-3','FG% from 3-10','FG% from 10-16','FG% from 16-3P','FG% from 3P','2P%','3P%','FT%','True Shooting','OWS','DWS','WS/48','OBPM','DBPM','Height','Weight','FG%_P1','Avg dist_P1','% of shots 2P_P1','% of shots 0-3_P1','% of shots 3-10_P1','% of shots 10-16_P1','% of shots 16-3P_P1','% of shots 3P_P1','FG% from 2P_P1','FG% from 0-3_P1','FG% from 3-10_P1','FG% from 10-16_P1','FG% from 16-3P_P1','FG% from 3P_P1','% assisted 2P_P1','% assisted 3P_P1','% of 3P corner_P1','corner 3P %_P1','2P%_P1','3P%_P1','FT%_P1','True Shooting_P1','AST%_P1','OWS_P1','DWS_P1','WS/48_P1','OBPM_P1','DBPM_P1','Height_P1',
 'Weight_P1','FG%_P2','Avg dist_P2','% of shots 2P_P2','% of shots 0-3_P2','% of shots 3-10_P2','% of shots 10-16_P2','% of shots 16-3P_P2','% of shots 3P_P2','FG% from 2P_P2','FG% from 0-3_P2','FG% from 3-10_P2','FG% from 10-16_P2','FG% from 16-3P_P2','FG% from 3P_P2','% assisted 2P_P2','% assisted 3P_P2','% of 3P corner_P2','corner 3P %_P2','2P%_P2','3P%_P2','FT%_P2','True Shooting_P2','AST%_P2','OWS_P2','DWS_P2','WS/48_P2','OBPM_P2','DBPM_P2','Height_P2','Weight_P2','FG%_P3','Avg dist_P3','% of shots 2P_P3','% of shots 0-3_P3','% of shots 3-10_P3','% of shots 10-16_P3','% of shots 16-3P_P3','% of shots 3P_P3','FG% from 2P_P3','FG% from 0-3_P3','FG% from 3-10_P3','FG% from 10-16_P3','FG% from 16-3P_P3','FG% from 3P_P3','% assisted 2P_P3','% assisted 3P_P3','% of 3P corner_P3','corner 3P %_P3','2P%_P3','3P%_P3','FT%_P3','True Shooting_P3','AST%_P3','OWS_P3','DWS_P3','WS/48_P3','OBPM_P3','DBPM_P3','Height_P3','Weight_P3','FG%_P4',
 'Avg dist_P4','% of shots 2P_P4','% of shots 0-3_P4','% of shots 3-10_P4','% of shots 10-16_P4','% of shots 16-3P_P4','% of shots 3P_P4','FG% from 2P_P4','FG% from 0-3_P4','FG% from 3-10_P4','FG% from 10-16_P4','FG% from 16-3P_P4','FG% from 3P_P4','% assisted 2P_P4','% assisted 3P_P4','% of 3P corner_P4','corner 3P %_P4','2P%_P4','3P%_P4','FT%_P4','True Shooting_P4','AST%_P4','OWS_P4','DWS_P4','WS/48_P4','OBPM_P4','DBPM_P4','Height_P4','Weight_P4','FG%_P5','Avg dist_P5','% of shots 2P_P5','% of shots 0-3_P5','% of shots 3-10_P5','% of shots 10-16_P5','% of shots 16-3P_P5','% of shots 3P_P5','FG% from 2P_P5','FG% from 0-3_P5','FG% from 3-10_P5','FG% from 10-16_P5','FG% from 16-3P_P5','FG% from 3P_P5','% assisted 2P_P5','% assisted 3P_P5','% of 3P corner_P5','corner 3P %_P5','2P%_P5','3P%_P5','FT%_P5','True Shooting_P5','AST%_P5','OWS_P5','DWS_P5','WS/48_P5','OBPM_P5','DBPM_P5','Height_P5',
 'Weight_P5','FG%_P6','Avg dist_P6','% of shots 2P_P6','% of shots 0-3_P6','% of shots 3-10_P6','% of shots 10-16_P6','% of shots 16-3P_P6','% of shots 3P_P6','FG% from 2P_P6','FG% from 0-3_P6','FG% from 3-10_P6','FG% from 10-16_P6','FG% from 16-3P_P6','FG% from 3P_P6','% assisted 2P_P6','% assisted 3P_P6','% of 3P corner_P6','corner 3P %_P6','2P%_P6','3P%_P6','FT%_P6','True Shooting_P6','AST%_P6','OWS_P6','DWS_P6','WS/48_P6','OBPM_P6','DBPM_P6','Height_P6','Weight_P6','FG%_P7','Avg dist_P7','% of shots 2P_P7','% of shots 0-3_P7','% of shots 3-10_P7','% of shots 10-16_P7','% of shots 16-3P_P7','% of shots 3P_P7','FG% from 2P_P7','FG% from 0-3_P7','FG% from 3-10_P7','FG% from 10-16_P7','FG% from 16-3P_P7','FG% from 3P_P7','% assisted 2P_P7','% assisted 3P_P7','% of 3P corner_P7','corner 3P %_P7','2P%_P7','3P%_P7','FT%_P7','True Shooting_P7','AST%_P7','OWS_P7','DWS_P7','WS/48_P7','OBPM_P7','DBPM_P7','Height_P7','Weight_P7','FG%_P8','Avg dist_P8','% of shots 2P_P8','% of shots 0-3_P8','% of shots 3-10_P8','% of shots 10-16_P8','% of shots 16-3P_P8','% of shots 3P_P8','FG% from 2P_P8','FG% from 0-3_P8','FG% from 3-10_P8','FG% from 10-16_P8','FG% from 16-3P_P8','FG% from 3P_P8','% assisted 2P_P8','% assisted 3P_P8','% of 3P corner_P8','corner 3P %_P8','2P%_P8','3P%_P8','FT%_P8','True Shooting_P8','AST%_P8','OWS_P8','DWS_P8','WS/48_P8','OBPM_P8','DBPM_P8','Height_P8',
 'Weight_P8','FG%_P9','Avg dist_P9','% of shots 2P_P9','% of shots 0-3_P9','% of shots 3-10_P9','% of shots 10-16_P9','% of shots 16-3P_P9','% of shots 3P_P9','FG% from 2P_P9','FG% from 0-3_P9','FG% from 3-10_P9','FG% from 10-16_P9','FG% from 16-3P_P9','FG% from 3P_P9','% assisted 2P_P9','% assisted 3P_P9','% of 3P corner_P9','corner 3P %_P9','2P%_P9','3P%_P9','FT%_P9','True Shooting_P9','AST%_P9','OWS_P9','DWS_P9','WS/48_P9','OBPM_P9','DBPM_P9','Height_P9','Weight_P9','FG%_P10','Avg dist_P10','% of shots 2P_P10','% of shots 0-3_P10','% of shots 3-10_P10','% of shots 10-16_P10','% of shots 16-3P_P10','% of shots 3P_P10','FG% from 2P_P10','FG% from 0-3_P10','FG% from 3-10_P10','FG% from 10-16_P10','FG% from 16-3P_P10','FG% from 3P_P10','% assisted 2P_P10','% assisted 3P_P10','% of 3P corner_P10','corner 3P %_P10','2P%_P10','3P%_P10','FT%_P10','True Shooting_P10','AST%_P10','OWS_P10','DWS_P10','WS/48_P10','OBPM_P10','DBPM_P10','Height_P10','Weight_P10']
].copy()
numericstat

,FG%,Avg dist,% of shots 2P,% of shots 0-3,% of shots 3-10,% of shots 10-16,% of shots 16-3P,% of shots 3P,FG% from 2P,FG% from 0-3,...,FT%_P10,True Shooting_P10,AST%_P10,OWS_P10,DWS_P10,WS/48_P10,OBPM_P10,DBPM_P10,Height_P10,Weight_P10
0,0.434,16.2,0.590,0.155,0.096,0.155,0.185,0.410,0.487,0.593,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
1,0.476,9.8,0.768,0.377,0.214,0.086,0.090,0.232,0.513,0.638,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
2,0.401,15.8,0.543,0.226,0.099,0.080,0.138,0.457,0.449,0.600,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
3,0.538,10.2,0.963,0.288,0.211,0.111,0.352,0.037,0.547,0.748,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
4,0.514,2.4,0.998,0.687,0.290,0.019,0.003,0.002,0.515,0.600,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,0.410,19.5,0.262,0.153,0.043,0.013,0.053,0.738,0.534,0.650,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5536,0.504,9.2,0.953,0.369,0.129,0.183,0.272,0.047,0.515,0.679,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5537,0.504,9.2,0.953,0.369,0.129,0.183,0.272,0.047,0.515,0.679,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5538,0.434,12.1,0.758,0.271,0.197,0.119,0.171,0.242,0.477,0.615,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040


In [19]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import linear_model

#df_mice = df.filter(['Distance','MaxSpeed','AvgSpeed','AvgMovingSpeed'], axis=1).copy()

# Define MICE Imputer and fill missing values
stat_imputer = IterativeImputer(estimator=linear_model.BayesianRidge(), n_nearest_features=None, imputation_order='ascending')

stats_mice_imputed = pd.DataFrame(stat_imputer.fit_transform(numericstat), columns= numericstat.columns)
stats_mice_imputed

C:\Users\Acer\anaconda3\envs\pythonProject3\lib\site-packages\sklearn\impute\_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,FG%,Avg dist,% of shots 2P,% of shots 0-3,% of shots 3-10,% of shots 10-16,% of shots 16-3P,% of shots 3P,FG% from 2P,FG% from 0-3,...,FT%_P10,True Shooting_P10,AST%_P10,OWS_P10,DWS_P10,WS/48_P10,OBPM_P10,DBPM_P10,Height_P10,Weight_P10
0,0.434,16.2,0.590,0.155,0.096,0.155,0.185,0.410,0.487,0.593,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
1,0.476,9.8,0.768,0.377,0.214,0.086,0.090,0.232,0.513,0.638,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
2,0.401,15.8,0.543,0.226,0.099,0.080,0.138,0.457,0.449,0.600,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
3,0.538,10.2,0.963,0.288,0.211,0.111,0.352,0.037,0.547,0.748,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
4,0.514,2.4,0.998,0.687,0.290,0.019,0.003,0.002,0.515,0.600,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,0.410,19.5,0.262,0.153,0.043,0.013,0.053,0.738,0.534,0.650,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5536,0.504,9.2,0.953,0.369,0.129,0.183,0.272,0.047,0.515,0.679,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5537,0.504,9.2,0.953,0.369,0.129,0.183,0.272,0.047,0.515,0.679,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5538,0.434,12.1,0.758,0.271,0.197,0.119,0.171,0.242,0.477,0.615,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040


In [20]:
stat_and_impute = allstat.merge(stats_mice_imputed, left_index=True, right_index=True,
                 how='outer', suffixes=('', '_imp'))

stat_and_impute

#dfNew.drop(dfNew.filter(regex='_y$').columns, axis=1, inplace=True)

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Game_ID,Event_Number,Frame_Number,game_clock_x,shot_clock_x,quarter_x,...,FT%_P10_imp,True Shooting_P10_imp,AST%_P10_imp,OWS_P10_imp,DWS_P10_imp,WS/48_P10_imp,OBPM_P10_imp,DBPM_P10_imp,Height_P10_imp,Weight_P10_imp
0,0,0,0,0,21500001,5,315,689.51,16.02,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
1,1,1,1,1,21500001,6,320,670.46,15.60,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
2,2,2,2,2,21500001,7,296,650.49,16.09,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
3,3,3,3,3,21500001,8,296,635.54,15.10,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
4,4,5,5,5,21500001,10,142,620.75,18.71,1,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,5535,16797,16797,16797,21500151,452,119,622.14,11.43,4,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5536,5536,16800,16800,16800,21500151,466,108,557.10,14.53,4,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5537,5537,16801,16801,16801,21500151,466,216,552.78,10.00,4,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5538,5538,16802,16802,16802,21500151,467,108,538.11,12.71,4,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040


In [23]:
only_imped_stat = stat_and_impute.drop(columns = ['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0','FG%','Avg dist','% of shots 2P','% of shots 0-3','% of shots 3-10','% of shots 10-16','% of shots 16-3P','% of shots 3P','FG% from 2P','FG% from 0-3','FG% from 3-10','FG% from 10-16','FG% from 16-3P','FG% from 3P','2P%','3P%','FT%','True Shooting','OWS','DWS','WS/48','OBPM','DBPM','Height','Weight','FG%_P1','Avg dist_P1','% of shots 2P_P1','% of shots 0-3_P1','% of shots 3-10_P1','% of shots 10-16_P1','% of shots 16-3P_P1','% of shots 3P_P1','FG% from 2P_P1','FG% from 0-3_P1','FG% from 3-10_P1','FG% from 10-16_P1','FG% from 16-3P_P1','FG% from 3P_P1','% assisted 2P_P1','% assisted 3P_P1','% of 3P corner_P1','corner 3P %_P1','2P%_P1','3P%_P1','FT%_P1','True Shooting_P1','AST%_P1','OWS_P1','DWS_P1','WS/48_P1','OBPM_P1','DBPM_P1','Height_P1',
 'Weight_P1','FG%_P2','Avg dist_P2','% of shots 2P_P2','% of shots 0-3_P2','% of shots 3-10_P2','% of shots 10-16_P2','% of shots 16-3P_P2','% of shots 3P_P2','FG% from 2P_P2','FG% from 0-3_P2','FG% from 3-10_P2','FG% from 10-16_P2','FG% from 16-3P_P2','FG% from 3P_P2','% assisted 2P_P2','% assisted 3P_P2','% of 3P corner_P2','corner 3P %_P2','2P%_P2','3P%_P2','FT%_P2','True Shooting_P2','AST%_P2','OWS_P2','DWS_P2','WS/48_P2','OBPM_P2','DBPM_P2','Height_P2','Weight_P2','FG%_P3','Avg dist_P3','% of shots 2P_P3','% of shots 0-3_P3','% of shots 3-10_P3','% of shots 10-16_P3','% of shots 16-3P_P3','% of shots 3P_P3','FG% from 2P_P3','FG% from 0-3_P3','FG% from 3-10_P3','FG% from 10-16_P3','FG% from 16-3P_P3','FG% from 3P_P3','% assisted 2P_P3','% assisted 3P_P3','% of 3P corner_P3','corner 3P %_P3','2P%_P3','3P%_P3','FT%_P3','True Shooting_P3','AST%_P3','OWS_P3','DWS_P3','WS/48_P3','OBPM_P3','DBPM_P3','Height_P3','Weight_P3','FG%_P4',
 'Avg dist_P4','% of shots 2P_P4','% of shots 0-3_P4','% of shots 3-10_P4','% of shots 10-16_P4','% of shots 16-3P_P4','% of shots 3P_P4','FG% from 2P_P4','FG% from 0-3_P4','FG% from 3-10_P4','FG% from 10-16_P4','FG% from 16-3P_P4','FG% from 3P_P4','% assisted 2P_P4','% assisted 3P_P4','% of 3P corner_P4','corner 3P %_P4','2P%_P4','3P%_P4','FT%_P4','True Shooting_P4','AST%_P4','OWS_P4','DWS_P4','WS/48_P4','OBPM_P4','DBPM_P4','Height_P4','Weight_P4','FG%_P5','Avg dist_P5','% of shots 2P_P5','% of shots 0-3_P5','% of shots 3-10_P5','% of shots 10-16_P5','% of shots 16-3P_P5','% of shots 3P_P5','FG% from 2P_P5','FG% from 0-3_P5','FG% from 3-10_P5','FG% from 10-16_P5','FG% from 16-3P_P5','FG% from 3P_P5','% assisted 2P_P5','% assisted 3P_P5','% of 3P corner_P5','corner 3P %_P5','2P%_P5','3P%_P5','FT%_P5','True Shooting_P5','AST%_P5','OWS_P5','DWS_P5','WS/48_P5','OBPM_P5','DBPM_P5','Height_P5',
 'Weight_P5','FG%_P6','Avg dist_P6','% of shots 2P_P6','% of shots 0-3_P6','% of shots 3-10_P6','% of shots 10-16_P6','% of shots 16-3P_P6','% of shots 3P_P6','FG% from 2P_P6','FG% from 0-3_P6','FG% from 3-10_P6','FG% from 10-16_P6','FG% from 16-3P_P6','FG% from 3P_P6','% assisted 2P_P6','% assisted 3P_P6','% of 3P corner_P6','corner 3P %_P6','2P%_P6','3P%_P6','FT%_P6','True Shooting_P6','AST%_P6','OWS_P6','DWS_P6','WS/48_P6','OBPM_P6','DBPM_P6','Height_P6','Weight_P6','FG%_P7','Avg dist_P7','% of shots 2P_P7','% of shots 0-3_P7','% of shots 3-10_P7','% of shots 10-16_P7','% of shots 16-3P_P7','% of shots 3P_P7','FG% from 2P_P7','FG% from 0-3_P7','FG% from 3-10_P7','FG% from 10-16_P7','FG% from 16-3P_P7','FG% from 3P_P7','% assisted 2P_P7','% assisted 3P_P7','% of 3P corner_P7','corner 3P %_P7','2P%_P7','3P%_P7','FT%_P7','True Shooting_P7','AST%_P7','OWS_P7','DWS_P7','WS/48_P7','OBPM_P7','DBPM_P7','Height_P7','Weight_P7','FG%_P8','Avg dist_P8','% of shots 2P_P8','% of shots 0-3_P8','% of shots 3-10_P8','% of shots 10-16_P8','% of shots 16-3P_P8','% of shots 3P_P8','FG% from 2P_P8','FG% from 0-3_P8','FG% from 3-10_P8','FG% from 10-16_P8','FG% from 16-3P_P8','FG% from 3P_P8','% assisted 2P_P8','% assisted 3P_P8','% of 3P corner_P8','corner 3P %_P8','2P%_P8','3P%_P8','FT%_P8','True Shooting_P8','AST%_P8','OWS_P8','DWS_P8','WS/48_P8','OBPM_P8','DBPM_P8','Height_P8',
 'Weight_P8','FG%_P9','Avg dist_P9','% of shots 2P_P9','% of shots 0-3_P9','% of shots 3-10_P9','% of shots 10-16_P9','% of shots 16-3P_P9','% of shots 3P_P9','FG% from 2P_P9','FG% from 0-3_P9','FG% from 3-10_P9','FG% from 10-16_P9','FG% from 16-3P_P9','FG% from 3P_P9','% assisted 2P_P9','% assisted 3P_P9','% of 3P corner_P9','corner 3P %_P9','2P%_P9','3P%_P9','FT%_P9','True Shooting_P9','AST%_P9','OWS_P9','DWS_P9','WS/48_P9','OBPM_P9','DBPM_P9','Height_P9','Weight_P9','FG%_P10','Avg dist_P10','% of shots 2P_P10','% of shots 0-3_P10','% of shots 3-10_P10','% of shots 10-16_P10','% of shots 16-3P_P10','% of shots 3P_P10','FG% from 2P_P10','FG% from 0-3_P10','FG% from 3-10_P10','FG% from 10-16_P10','FG% from 16-3P_P10','FG% from 3P_P10','% assisted 2P_P10','% assisted 3P_P10','% of 3P corner_P10','corner 3P %_P10','2P%_P10','3P%_P10','FT%_P10','True Shooting_P10','AST%_P10','OWS_P10','DWS_P10','WS/48_P10','OBPM_P10','DBPM_P10','Height_P10','Weight_P10'
])
only_imped_stat

,Game_ID,Event_Number,Frame_Number,game_clock_x,shot_clock_x,quarter_x,b_x_x,b_y_x,p1_team_id_x,p1_player_id_x,...,FT%_P10_imp,True Shooting_P10_imp,AST%_P10_imp,OWS_P10_imp,DWS_P10_imp,WS/48_P10_imp,OBPM_P10_imp,DBPM_P10_imp,Height_P10_imp,Weight_P10_imp
0,21500001,5,315,689.51,16.02,1,25.76351,18.34473,1.610613e+09,2594.0,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
1,21500001,6,320,670.46,15.60,1,62.25751,9.78790,1.610613e+09,2594.0,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
2,21500001,7,296,650.49,16.09,1,17.46303,47.71408,1.610613e+09,2594.0,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
3,21500001,8,296,635.54,15.10,1,69.76106,15.08616,1.610613e+09,2594.0,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
4,21500001,10,142,620.75,18.71,1,10.38510,37.43310,1.610613e+09,2594.0,...,0.389,0.504,3.9,3.4,4.3,0.147,0.6,-0.7,210.82,126.552168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,21500151,452,119,622.14,11.43,4,19.27161,17.85623,1.610613e+09,977.0,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5536,21500151,466,108,557.10,14.53,4,65.88512,14.40700,1.610613e+09,977.0,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5537,21500151,466,216,552.78,10.00,4,86.54179,32.19858,1.610613e+09,977.0,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040
5538,21500151,467,108,538.11,12.71,4,32.66729,28.37212,1.610613e+09,977.0,...,0.679,0.436,10.5,-1.4,1.8,0.012,-4.4,1.4,200.66,111.130040


In [24]:
only_imped_stat.to_csv(r'D:\Szakdolgozat_data\Features_jupyter\only_imped_stat.csv')

In [5]:
df2 = pd.read_csv(r'D:\Szakdolgozat_data\Features_jupyter\only_imped_stat.csv')
list(df2.columns)

['Unnamed: 0',
 'Game_ID',
 'Event_Number',
 'Frame_Number',
 'game_clock_x',
 'shot_clock_x',
 'quarter_x',
 'b_x_x',
 'b_y_x',
 'p1_team_id_x',
 'p1_player_id_x',
 'p1_x_x',
 'p1_y_x',
 'p2_team_id_x',
 'p2_player_id_x',
 'p2_x_x',
 'p2_y_x',
 'p3_team_id_x',
 'p3_player_id_x',
 'p3_x_x',
 'p3_y_x',
 'p4_team_id_x',
 'p4_player_id_x',
 'p4_x_x',
 'p4_y_x',
 'p5_team_id_x',
 'p5_player_id_x',
 'p5_x_x',
 'p5_y_x',
 'p6_team_id_x',
 'p6_player_id_x',
 'p6_x_x',
 'p6_y_x',
 'p7_team_id_x',
 'p7_player_id_x',
 'p7_x_x',
 'p7_y_x',
 'p8_team_id_x',
 'p8_player_id_x',
 'p8_x_x',
 'p8_y_x',
 'p9_team_id_x',
 'p9_player_id_x',
 'p9_x_x',
 'p9_y_x',
 'p10_team_id_x',
 'p10_player_id_x',
 'p10_x_x',
 'p10_y_x',
 'game_clock_plus25',
 'shot_clock_plus25',
 'b_x_plus25',
 'b_y_plus25',
 'p1_x_plus25',
 'p1_y_plus25',
 'p2_x_plus25',
 'p2_y_plus25',
 'p3_x_plus25',
 'p3_y_plus25',
 'p4_x_plus25',
 'p4_y_plus25',
 'p5_x_plus25',
 'p5_y_plus25',
 'p6_x_plus25',
 'p6_y_plus25',
 'p7_x_plus25',
 'p7_

In [6]:
df2['Shooter_Team_Name'] = 'no team' 
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612737 , 'Atlanta Hawks',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612738 , 'Boston Celtics',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612739 , 'Brooklyn Nets',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612740 , 'Charlotte Hornets',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612741 , 'Chicago Bulls',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612742 , 'Cleveland Cavaliers',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612743 , 'Dallas Mavericks',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612744 , 'Denver Nuggets',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612745 , 'Detroit Pistons',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612746 , 'Golden State Warriors',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612747 , 'Houston Rockets',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612748 , 'Indiana Pacers',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612749 , 'Los Angeles Clippers',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612750 , 'Los Angeles Lakers',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612751 , 'Memphis Grizzlies',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612752 , 'Miami Heat',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612753 , 'Milwaukee Bucks',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612754 , 'Minnesota Timberwolves',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612755 , 'New Orleans Pelicans',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612756 , 'New York Knicks',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612757 , 'Oklahoma City Thunder',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612758 , 'Orlando Magic',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612759 , 'Philadelphia 76ers',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612760 , 'Phoenix Suns',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612761 , 'Portland Trail Blazers',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612762 , 'Sacramento Kings',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612763 , 'San Antonio Spurs',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612764 , 'Toronto Raptors',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612765, 'Utah Jazz',df2['Shooter_Team_Name'])
df2['Shooter_Team_Name'] = np.where(df2['SHOOTER_TEAM_ID']==1610612766 , 'Washington Wizards',df2['Shooter_Team_Name'])


In [8]:
df2.to_csv(r'D:\Szakdolgozat_data\Features_jupyter\df2.csv')

In [3]:
# visit http://127.0.0.1:8050/ in your web browser.



Dash app running on http://127.0.0.1:8050/


In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


app = JupyterDash(__name__)


fig = px.scatter(df2, x="FG%_imp", y="SHOT_DISTANCE",
                  color="SHOOTER_TEAM_ID", hover_name="SHOOTER_NAME",
                 log_x=True, size_max=60)

app.layout = html.Div([
    dcc.Graph(
        id='distance_and_makes',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
#---------------------------------------Ez alatt a készek------------------------------------------

In [3]:
app = JupyterDash(__name__)

#fig = px.bar(df_bar, x="Fruit", y="Amount", color="City", barmode="group")


team_dropdown = dcc.Dropdown(options=df2['SHOOTER_TEAM_ID'].unique())
player_dropdown = dcc.Dropdown(options=df2['SHOOTER_NAME'].unique())

app.layout = html.Div([
    html.Div(children=[
    html.H1(children='Team shots graph'),
    team_dropdown,
    dcc.Graph(id='team_shot-graph')  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div(children=[
    html.H1(children='Player graph'),
    player_dropdown,
    dcc.Graph(id='player-graph')  
    ])
])
# Callback function that automatically updates the tip-graph based on chosen colorscale
@app.callback(
    Output(component_id='team_shot-graph', component_property='figure'),
    Output(component_id='player-graph', component_property='figure'),
    Input(component_id=team_dropdown, component_property='value'),
    Input(component_id=player_dropdown, component_property='value')
)

def update_graph(selected_team, selected_player):
    filtered_df2 = df2[df2['SHOOTER_TEAM_ID'] == selected_team]
    filtered_df3 = df2[df2['SHOOTER_NAME'] == selected_player]
    fig = px.scatter(filtered_df2,
                       x='SHOT_DISTANCE', y='FG%_imp',
                       color='SHOT_MADE_FLAG', hover_name="SHOOTER_NAME", color_continuous_scale=["red", "lightgreen"],
                       title=f'Shots of team: {selected_team}')
    fig2 = px.scatter(filtered_df3,
                       x='SHOT_DISTANCE', y='CLOSEST_DEFENDER_DIST',
                       color='SHOT_MADE_FLAG', color_continuous_scale=["red", "lightgreen"],
                       title=f'Shots of {selected_player}')
    return fig, fig2

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import pandas as pd

app = JupyterDash(__name__)

#df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
team_dropdown = dcc.Dropdown(options=df2['SHOOTER_TEAM_ID'].unique())

app.layout = html.Div(children=[
    html.H1(children='Team shots graph'),
    team_dropdown,
    dcc.Graph(id='team_shot-graph')
])
@app.callback(
    Output(component_id='team_shot-graph', component_property='figure'),
    Input(component_id=team_dropdown, component_property='value')
)
def update_graph(selected_team):
    filtered_df2 = df2[df2['SHOOTER_TEAM_ID'] == selected_team]
    fig = px.scatter(filtered_df2,
                       x='SHOT_DISTANCE', y='FG%_imp',
                       color='SHOT_MADE_FLAG', hover_name="SHOOTER_NAME",
                       title=f'Shots of team: {selected_team}')
    return fig



if __name__ == '__main__':
    app.run_server(debug=True)
